<a href="https://colab.research.google.com/github/keithk1210/OCR-Fall-2023/blob/main/Cherokee_OCR_Fall_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependencies

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

In [ ]:
import cv2
import numpy as np
import osgeo_utils
import pytesseract
import os
import re
import csv


from PIL import ImageFont, ImageDraw, Image
from langdetect import detect
from pytesseract import Output
from google.colab.patches import cv2_imshow
from imutils.object_detection import non_max_suppression

In [ ]:
!mkdir tessdata
!wget -O ./tessdata/chr.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/chr.traineddata?raw=true
!wget -O ./tessdata/eng.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/eng.traineddata?raw=true

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def bounding_box(result, draw, i, color = (255,100,0)):
  x = result['left'][i]
  y = result['top'][i]
  w = result['width'][i]
  h = result['height'][i]

  draw.rectangle((x, y, x + w, y + h), outline=color, width=2)

In [ ]:
while True:
  input = input("Show output? (Y/N) ")
  show_output = -1
  if input.upper() == 'Y':
    show_output = True
    break
  elif input.upper() == 'N':
    show_output = False
    break
  else:
    print("Incorrect input, try again")
dir = input("Directory: ")
csv_file_path = input("CSV file path: ")

In [ ]:

config_tesseract = "--tessdata-dir tessdata --psm 3 chr"
csv_file_path = '/content/drive/MyDrive/OCR_Output/service-ll-llscd-14021571-14021571.csv - Sheet1.csv'
data = [
    ['Filename','Cherokee']
]
original_images = []
images = []
j = 0
dir_len = len(os.listdir(dir))
for image in os.listdir(dir):
  print(j, "/", dir_len)
  img = cv2.imread(dir + "/" + image)
  original = img.copy()
  H = img.shape[0]
  W = img.shape[1]
  width, height = 320, 320
  proportion_W = W / float(width)
  proportion_H = H / float(height)

  value, img = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)
  if show_output:
    cv2_imshow(img)
  original_images.append(img)
  #img = cv2.resize(img, (width, height))
  H = img.shape[0]
  W = img.shape[1]
  images.append(img)
  min_confidence = 40
  img_copy = img.copy()
  cv2_im_rgb = cv2.cvtColor(img_copy,cv2.COLOR_BGR2RGB)
  pil_im = Image.fromarray(cv2_im_rgb)
  draw = ImageDraw.Draw(pil_im)
  font = ImageFont.truetype("/content/drive/MyDrive/NotoSansCherokee-VariableFont_wght.ttf",20)
  result = pytesseract.image_to_data(img, config=config_tesseract, lang='chr', output_type=Output.DICT)
  chr_text = ''
  for i in range(0, len(result['text'])):
    confidence = int(result['conf'][i])
    if confidence > min_confidence:
      x, y = bounding_box(result, draw, i)
      text = result['text'][i]
      draw.text((x, y-20), text, font=font,fill=(0, 0, 255, 255))
      if not text.isdigit():
        chr_text += text
  data.append([image,chr_text])
  cv2_im_processed = cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)
  #cv2_imshow(cv2_im_processed)
  j += 1
print(data)
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(data)


